# Chapter 3:
# Multi-Armed Bandits: Evaluate multiple system changes while maximizing business metrics

In [ ]:
import numpy as np
import scipy
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt
from e4e import E4E

e4e = E4E(chapter=3)

## 3.1	Epsilon-greedy: Account for the impact of evaluation on business metrics

### 3.1.1	A/B testing as a baseline

In [ ]:
# Listing 3.1 Simulate measurement of click on an ad
def measure_click(ctr):
    return 1 if np.random.uniform(0,1) < ctr else 0

def measure_a():
    return measure_click(ctr=0.0050)

def measure_b():
    return measure_click(ctr=0.0070)

In [ ]:
# Listing 3.2 Design an A/B test
def design_ab_test():
    def pilot_study(num_pilot_measurements):
        clicked_pre_a = np.array([measure_a() for _ in range(num_pilot_measurements)])
        clicked_pre_b = np.array([measure_b() for _ in range(num_pilot_measurements)])
        sd_1 = np.sqrt( clicked_pre_a.std()**2 + clicked_pre_b.std()**2 )  
        return sd_1
    
    sd_1 = pilot_study(1000)
    prac_sig = 0.001    
    num_ind = (2.48 * sd_1 / prac_sig) ** 2
    return int(num_ind)

In [ ]:
# Listing 3.3 Simulate a run of an A/B test
def run_ab_test(num_ind):
    clicked_a = []
    clicked_b = []
    for n in range(num_ind):
        # Randomize between A and B.
        if np.random.uniform(0,1) < 0.5:
            clicked = measure_a()
            clicked_a.append(clicked)
        else:
            clicked = measure_b()
            clicked_b.append(clicked)

    clicked_a = np.array(clicked_a)
    clicked_b = np.array(clicked_b)
    
    return clicked_a, clicked_b

In [ ]:
# Listing 3.4 Analyze the A/B test data
def analyze_a_b_test(clicked_a, clicked_b):   
    mean_a = clicked_a.mean()
    mean_b = clicked_b.mean()
    std_a = clicked_a.std()
    std_b = clicked_b.std()
    m = mean_b - mean_a
    se = np.sqrt( (std_a**2 + std_b**2) / num_ind )
    z = m / se
    
    return z

In [ ]:
np.random.seed(17)
num_ind = design_ab_test()
clicked_A, clicked_B = run_ab_test(num_ind)
z = analyze_a_b_test(clicked_A, clicked_B)
print (num_ind, z)

In [ ]:
# Listing 3.5 Trace the CTR as the A/B test runs
def ab_test(num_ind):
    sum_clicks = 0.0
    num_ads = 0.0
    sum_a = 0.0
    num_a = 0
    sum_b = 0.0
    num_b = 0

    ctr_vs_n = []
    ctr_a = []
    ctr_b = []
    for n in range(num_ind):
        if np.random.uniform(0,1) < 0.5:
            clicked = measure_a()
            sum_a += clicked
            num_a += 1
        else:
            clicked = measure_b()
            sum_b += clicked
            num_b += 1
        sum_clicks += clicked
        num_ads += 1
        if num_a > 0 and num_b > 0:
            ctr_a.append(sum_a/num_a)
            ctr_b.append(sum_b/num_b)
            ctr_vs_n.append(sum_clicks/num_ads)            
    
    return ctr_vs_n, ctr_a, ctr_b

In [ ]:
np.random.seed(17)
ctr_vs_n, ctr_a, ctr_b = ab_test(num_ind)

In [ ]:
plt.plot(ctr_vs_n, '--', color=e4e.color_1);
plt.xlabel('n, index to individual measurement')
plt.ylabel('CTR up through n');
e4e.save_fig(3)

In [ ]:
def run_multi(trace_fn, num=100):
    traces = []
    n = 1e99
    for _ in range(num):
        trace = trace_fn()
        n = min(n, len(trace))
        traces.append(trace)

    traces_aligned = []
    for t in traces:
        traces_aligned.append(t[-n:])
    traces = np.array(traces_aligned)

    means = traces.mean(axis=0)
    stds = traces.std(axis=0)
    
    return means, stds

In [ ]:
np.random.seed(17)
means_ab, stds_ab = run_multi(lambda: ab_test(num_ind)[0], 100)
print (means_ab[-1], stds_ab[-1])

In [ ]:
n = np.arange(len(means_ab))[::100]
plt.fill_between(n,
                 (means_ab-stds_ab/2)[::100],
                 (means_ab+stds_ab/2)[::100],
                 color=e4e.color_3, alpha=.75, linewidth=1)

plt.plot(n, means_ab[::100], '-', color=e4e.color_1)
c = plt.axis()
plt.xlabel('n, index to individual measurement')
plt.ylabel('CTR up through n');
plt.axis([c[0], c[1], .0040, .0080]);

e4e.save_fig(4)

In [ ]:
plt.plot(ctr_a, '-', color=e4e.color_1);
plt.plot(ctr_b, '--', color=e4e.color_2);
print (ctr_a[-1], ctr_b[-1])
plt.xlabel('n, index to individual measurement')
plt.ylabel('CTR up through n');
plt.legend(['CTR A', 'CTR B'])
plt.annotate("CTR A > CTR B", xy=[10000, .0080],
             xytext=[50000, .015],
             arrowprops=e4e.arrow_props
            )
plt.annotate("CTR A < CTR B", xy=[80000, .0070],
             xytext=[70000, .010],
             arrowprops=e4e.arrow_props
            )
e4e.save_fig(5)

### 3.1.2	The epsilon-greedy algorithm

In [ ]:
# Listing 3.6 The epsilon-greedy algorithm
def epsilon_greedy(num_ind, epsilon):
    sum_clicks = 0.0
    num_ads = 0.0
    sum_a = 0.0
    num_a = 0
    sum_b = 0.0
    num_b = 0
    ctr_vs_n = []
    used_b = []
    for _ in range(int(num_ind)):
        select = "Randomize"
        if np.random.uniform(0,1) < 1-epsilon:
            ctr_a = sum_a/num_a if num_a>0 else 0
            ctr_b = sum_b/num_b if num_b>0 else 0
            if ctr_a > ctr_b:
                select = "A"
            elif ctr_b > ctr_a:
                select = "B"
            # else, if they're equal, randomize
            
        if select == "Randomize":
            if np.random.uniform(0,1) < 0.5:
                select = "A"
            else:
                select = "B"
                
        if select == "A":
            clicked = measure_a()
            sum_a += clicked
            num_a += 1
            used_b.append(False)
        else:
            clicked = measure_b()
            sum_b += clicked
            num_b += 1
            used_b.append(True)
        sum_clicks += clicked
        num_ads += 1
        
        ctr_vs_n.append(sum_clicks / num_ads)
    
    return ctr_vs_n, used_b

In [ ]:
np.random.seed(17)
ctr_eps_greedy = epsilon_greedy(num_ind=num_ind, epsilon=0.1)[0]
print (ctr_eps_greedy[-1])

In [ ]:
plt.plot(ctr_vs_n, '-', color=e4e.color_1);
plt.plot(ctr_eps_greedy, '--', color=e4e.color_2);
plt.xlabel('n, index to individual measurement')
plt.ylabel('CTR up through n');
plt.legend(['A/B test', 'epsilon-greedy'])
e4e.save_fig(7)

In [ ]:
np.random.seed(17)
means_eg, stds_eg = run_multi(lambda: epsilon_greedy(num_ind, .1)[0], 100)
print (means_eg[-1], stds_eg[-1])

In [ ]:
means_eg_tr = means_eg[-len(means_ab):]
stds_eg_tr = stds_eg[-len(stds_ab):]
n = np.arange(len(means_ab))[::100]

plt.fill_between(n,
                 (means_ab-stds_ab/2)[::100],
                 (means_ab+stds_ab/2)[::100],
                 color=e4e.color_3, alpha=.75, linewidth=1)
plt.plot(n, means_ab[::100], '--', color=e4e.color_1)

plt.fill_between(n,
                 (means_eg_tr-stds_eg_tr/2)[::100],
                 (means_eg_tr+stds_eg_tr/2)[::100],
                 color=e4e.color_2, alpha=.75, linewidth=1)
plt.plot(n, means_eg_tr[::100], '-', color=e4e.color_2)



c = plt.axis()
plt.xlabel('n, index to individual measurement')
plt.ylabel('CTR up through n');
plt.legend(['A/B test','epsilon-greedy'])
plt.axis([c[0], c[1], .0040, .0080]);
e4e.save_fig(8)

In [ ]:
def run_multi_selection_rates(selected_fn, num=100):
    selected = []
    for _ in range(num):
        selected.append(selected_fn())

    selected = np.array(selected)
    indices = np.unique(selected)
    rate_selected = []
    for i in indices:
        chis = selected==i
        rate_selected.append(chis.mean(axis=0))
    rate_selected = np.array(rate_selected)
    
    return indices, rate_selected

In [ ]:
indices, rate_selected = run_multi_selection_rates(lambda: epsilon_greedy(num_ind, .1)[-1], 100)
print (indices)

In [ ]:
n = np.arange(rate_selected.shape[1])
plt.plot(n[::10], 100*rate_selected[0,:][::10], '.', color=e4e.color_2);
plt.plot(n[::10], 100*rate_selected[1,:][::10], '.', color=e4e.color_3);
plt.legend(['A selected', 'B selected'], markerscale=3)
plt.ylabel('Percentage of runs')
plt.xlabel('n, index to individual measurement')
e4e.save_fig(9)

### 3.1.3	Deciding when to stop

In [ ]:
# Listing 3.6 Epsilon-greedy with decaying epsilon
def epsilon_greedy_decay():
    bm_max = 0.01
    prac_sig = 0.001
    c = 5
    
    epsilon_0 = 2*c*(bm_max/prac_sig)**2
    epsilon_stop = 0.01
    
    sum_clicks = 0.0
    num_ads = 0.0
    sum_a = 0.0
    num_a = 0
    sum_b = 0.0
    num_b = 0
    ctr_vs_n = []
    epsilons = []
    
    n = 0
    selected = None
    while True:
        epsilon = min(1.0, epsilon_0 / (1.0 + n))
        epsilons.append(epsilon)
        if epsilon < epsilon_stop:
            break
        select = "Randomize"
        if np.random.uniform(0,1) < 1-epsilon:
            ctr_a = sum_a/num_a if num_a>0 else 0
            ctr_b = sum_b/num_b if num_b>0 else 0
            if ctr_a > ctr_b:
                select = "A"
                selected = "A"
            elif ctr_b > ctr_a:
                select = "B"
                selected = "B"
        if select == "Randomize":
            if np.random.uniform(0,1) < 0.5:
                select = "A"
            else:
                select = "B"

        if select == "A":
            clicked = measure_a()
            sum_a += clicked
            num_a += 1
        else:
            clicked = measure_b()
            sum_b += clicked
            num_b += 1
        sum_clicks += clicked
        num_ads += 1
        
        ctr_vs_n.append(sum_clicks / num_ads)
        n += 1

    if selected == "B":
        accept_reject = "Accept"
    else:
        accept_reject = "Reject"
    return ctr_vs_n, epsilons, accept_reject

In [ ]:
np.random.seed(17)
ctr_eps_greedy_decay, epsilons, accept_reject = epsilon_greedy_decay()
print (len(ctr_eps_greedy_decay), ctr_eps_greedy_decay[-1], accept_reject)

In [ ]:
plt.semilogy(epsilons, '--', color=e4e.color_1);
plt.xlabel('n, index to individual measurement')
plt.ylabel('epsilon used at n');
e4e.horizontal_line(.01)
e4e.save_fig(10)
epsilons[-1]

#### FALSE POSITIVES, FALSE NEGATIVES

## 3.2	Evaluate multiple system changes simultaneously

In [ ]:
def run_multi_ragged(trace_fn, num=100):
    finals = []
    for _ in range(num):
        trace = trace_fn()
        finals.append(trace[-1])
    finals = np.array(finals)
    mean = finals.mean()
    std = finals.std()
    
    return mean, std

In [ ]:
np.random.seed(17)
mean_selected, std_selected = run_multi_ragged(lambda: [int(epsilon_greedy_decay()[-1] == "Accept")], 100)
print (mean_selected, std_selected)

In [ ]:
def epsilon_greedy_decay_compare(num_ind):
    # Run for N measurements ignoring epsilon_stop
    
    bm_max = 0.01
    prac_sig = 0.001
    
    epsilon_0 = 2*5*(bm_max/prac_sig)**2
    
    sum_clicks = 0.0
    num_ads = 0.0
    sum_a = 0.0
    num_a = 0
    sum_b = 0.0
    num_b = 0
    ctr_vs_n = []
    epsilons = []
    
    selected = None
    for n in range(num_ind):
        epsilon = min(1.0, epsilon_0 / (1.0 + n))
        epsilons.append(epsilon)
        select = "Randomize"
        if np.random.uniform(0,1) < 1-epsilon:
            ctr_a = sum_a/num_a if num_a>0 else 0
            ctr_b = sum_b/num_b if num_b>0 else 0
            if ctr_a > ctr_b:
                select = "A"
                selected = "A"
            elif ctr_b > ctr_a:
                select = "B"
                selected = "B"
            
            # else, if they're equal, randomize
            
        if select == "Randomize":
            if np.random.uniform(0,1) < 0.5:
                select = "A"
            else:
                select = "B"
                
        if select == "A":
            clicked = measure_a()
            sum_a += clicked
            num_a += 1
        else:
            clicked = measure_b()
            sum_b += clicked
            num_b += 1
        sum_clicks += clicked
        num_ads += 1
        
        ctr_vs_n.append(sum_clicks / num_ads)

    if selected == "B":
        accept_reject = "Accept"
    else:
        accept_reject = "Reject"
    return ctr_vs_n, epsilons, accept_reject

In [ ]:
np.random.seed(17)
means_eg_dc, stds_eg_dc = run_multi(lambda: epsilon_greedy_decay_compare(num_ind)[0], 100)
print (means_eg_dc[-1], stds_eg_dc[-1])

In [ ]:
np.random.seed(17)
mean_selected, std_selected = run_multi(lambda: [int(epsilon_greedy_decay_compare(num_ind)[-1]=="Accept")], 100)
print (mean_selected, std_selected)

In [ ]:
# Listing 3.8 Simulate several click models
def measure_arm(i_arm):
    return measure_click(ctr=.005 + i_arm*.002)

In [ ]:
# Listing 3.7 Epsilon-greedy with four arms
def epsilon_greedy_decay_multi():
    bm_max = 0.01
    prac_sig = 0.001
    k = 4
    c = 5
    
    epsilon_0 = k*c*(bm_max/prac_sig)**2
    epsilon_stop = 0.01
    
    sum_clicks = 0.0
    num_ads = 0.0
    sum_arm = [0.0]*k
    num_arm = [0.0]*k
    ctr_vs_n = []
    
    n = 0
    arms_selected = []
    while True:
        epsilon = min(1.0, epsilon_0 / (1.0 + n))
        if epsilon < epsilon_stop:
            break
        i_selected = None
        if np.random.uniform(0,1) < 1-epsilon:
            max_ctr = None
            for i in range(k):
                if num_arm[i] > 0:
                    ctr_arm = sum_arm[i] / num_arm[i]
                else:
                    ctr_arm = 0
                # break ties by randomizing
                ctr_arm += 1e-9 * np.random.normal()
                if max_ctr is None or ctr_arm > max_ctr:
                    max_ctr = ctr_arm
                    i_selected = i
            i_best_arm = i_selected
        else:
            i_selected = np.random.randint(k)
                
        arms_selected.append(i_selected)
        clicked = measure_arm(i_selected)
        sum_arm[i_selected] += clicked 
        num_arm[i_selected] += 1
        sum_clicks += clicked
        num_ads += 1
        
        ctr_vs_n.append(sum_clicks / num_ads)
        n += 1

    return ctr_vs_n, arms_selected

In [ ]:
np.random.seed(17)
ctr_epsilon_greedy_decay_multi, arms_selected = epsilon_greedy_decay_multi()
print (len(ctr_epsilon_greedy_decay_multi), ctr_epsilon_greedy_decay_multi[-1], arms_selected[-1])

In [ ]:
np.random.seed(17)
mean_egd, std_egd  = run_multi(lambda: epsilon_greedy_decay_multi()[0], 100)
print (mean_egd[-1], std_egd[-1])

In [ ]:
np.random.seed(17)
mean_selected, std_selected = run_multi(lambda: [int(epsilon_greedy_decay_multi()[-1][-1] == 3)], 100)
print (mean_selected, std_selected)

In [ ]:
plt.plot(ctr_epsilon_greedy_decay_multi, '--', color=e4e.color_1);
plt.xlabel('n, index to individual measurement')
plt.ylabel('CTR up through n');
e4e.save_fig(12)

In [ ]:
indices_multi, rate_selected_multi = run_multi_selection_rates(lambda: epsilon_greedy_decay_multi()[-1], 100)
print (indices)

In [ ]:
n = np.arange(rate_selected_multi.shape[1])[::10]
legend = []
for i in range(4):
    plt.plot(n, 100*rate_selected_multi[i,::10], '.', color=e4e.colors[i]);
    legend.append(f'Arm {i} selected')
plt.legend(legend, markerscale=3)
plt.ylabel('Percentage of runs')
plt.xlabel('n, index to individual measurement')

e4e.save_fig(13)

## 3.3 Thompson Sampling: A more efficient bandit algorithm

### 3.3.1	Estimating the probability that an arm is the best

In [ ]:
I_clicked = np.array([0,0,1,0,1,1,0,0,1,0])

In [ ]:
ctr = I_clicked.mean()
print (ctr)

In [ ]:
se = I_clicked.std() / np.sqrt(len(I_clicked))
print (se)

In [ ]:
# Listing 3.9 Generate a bootstrap sample from a data set
def bootstrap_sample(data):
    n = len(data)
    return data[np.random.randint(n, size=(n,))]

In [ ]:
np.random.seed(17)
print (bootstrap_sample(I_clicked))
print (bootstrap_sample(I_clicked))
print (bootstrap_sample(I_clicked))

In [ ]:
def replicate_means(data, num_replications):
    means = []
    for _ in range(num_replications):
        means.append(bootstrap_sample(data).mean())
    return np.array(means)

In [ ]:
np.random.seed(17)
ctrs = replicate_means(I_clicked, 1000)
print (ctrs.mean())
print (ctrs.std())

In [ ]:
np.random.seed(17)
plt.hist(replicate_means(I_clicked, 1000), 25, color=e4e.color_1);
plt.xlabel('CTR')
e4e.save_fig(14)

#### WORKING WITH FEW INDIVIDUAL MEASUREMENTS

In [ ]:
np.random.seed(17)
fig, axs = plt.subplots(2, 2)
i_axs = [(0,0), (0,1), (1,0), (1,1)]
letters = ["a", "b", "c", "d"]
num_measurements = 10
for i, i_ax in enumerate(i_axs):
    ax = axs[i_ax]
    ax.hist(replicate_means(np.random.binomial(n=1, p=.4, size=(int(num_measurements+.5),)),
                            10000), 25, color=e4e.color_1);
    for label in ax.get_xticklabels():
        label.set_fontsize(7)
    
    locs = ax.get_yticks()
    ax.set_yticklabels([""]*len(locs))
    num_measurements *= 10
    c = ax.axis()
    ax.text(.1*c[0] + .90*c[1], .1*c[2] + .9*c[3], f"({letters[i]})", fontsize=7)

e4e.save_fig(15)

#### PROBABILITY OF BEING THE BEST ARM

In [ ]:
np.random.seed(17)
I_clicked_1 = np.array([measure_click(ctr=.005) for _ in range(10000)])
I_clicked_2 = np.array([measure_click(ctr=.007) for _ in range(10000)])

In [ ]:
print (I_clicked_1.mean())
print (I_clicked_2.mean())

In [ ]:
plt.hist(replicate_means(I_clicked_1, 1000), 10, color=e4e.color_1, alpha=.75)
plt.hist(replicate_means(I_clicked_2, 1000), 10, color=e4e.color_2, alpha=.75);
plt.legend(['Arm 1', 'Arm 2'])
plt.xlabel('CTR')
e4e.save_fig(16)

In [ ]:
# Listing 3.10 Estimate the probability of each arm being the better arm
def estimate_pbest(I_clicked_1, I_clicked_2):
    counts = [0, 0]
    num_samples = 100
    for _ in range(num_samples):
        ctr_1 = bootstrap_sample(I_clicked_1).mean()
        ctr_2 = bootstrap_sample(I_clicked_2).mean()
        if ctr_1 > ctr_2:
            counts[0] += 1
        elif ctr_2 > ctr_1:
            counts[1] += 1
        else:
            pass
            
    p_best = np.array(counts)/num_samples
    return p_best

In [ ]:
np.random.seed(17)
estimate_pbest(I_clicked_1, I_clicked_2)

In [ ]:
# Listing 3.11 Estimate the probability that each arm is best
def estimate_pbest(I_clickeds):
    counts = [0] * len(I_clickeds)
    num_samples = 100
    for _ in range(num_samples):
        ctrs = [bootstrap_sample(I_clicked).mean() for I_clicked in I_clickeds]
        ctrs = np.array(ctrs)
        i = np.where(ctrs == ctrs.max())[0]
        if len(i)==1:
            counts[i[0]] += 1
            
    return np.array(counts)/num_samples

In [ ]:
np.random.seed(17)
I_clickeds = [None]*4
I_clickeds[0] = np.array([measure_click(ctr=.003) for _ in range(10000)])
I_clickeds[1] = np.array([measure_click(ctr=.005) for _ in range(10000)])
I_clickeds[2] = np.array([measure_click(ctr=.007) for _ in range(10000)])
I_clickeds[3] = np.array([measure_click(ctr=.009) for _ in range(10000)])
estimate_pbest(I_clickeds)

### 3.3.2	Randomized Probability Matching

In [ ]:
# Listing 3.12 Randomized probability matching
def rpm_select_arm(I_clickeds):
    ctrs = [bootstrap_sample(I_clicked).mean() for I_clicked in I_clickeds]
    ctrs = np.array(ctrs)
    i = np.where(ctrs == ctrs.max())[0]
    if len(i)!=1:
        return np.random.randint(len(I_clickeds))
    return i[0]

In [ ]:
rpm_select_arm(I_clickeds)

#### ONLINE BOOTSTRAP

In [ ]:
# Listing 3.13 Online Bootstrap
class OnlineBootstrap:
    def __init__(self, num_bs_means):
        self._sums = np.zeros(shape=(num_bs_means,))
        self._n = np.zeros(shape=(num_bs_means,))
        self._count = 0
        
    def append(self, clicked):
        i = np.where(np.random.randint(2, size=(len(self._n,))) == 0)[0]
        self._sums[i] += clicked
        self._n[i] += 1
        self._count += 1

    def CTR_estimate(self):
        i = np.random.randint(len(self._n))
        if self._n[i] == 0:
            return np.inf
        return self._sums[i] / self._n[i]
    
    def count(self):
        return self._count

In [ ]:
# Listing 3.14 Randomized probability matching with the online bootstrap
def rpm_select_arm_ob(obs):
    ctrs = [ob.CTR_estimate() for ob in obs]
    ctrs = np.array(ctrs)
    i = np.where(ctrs == ctrs.max())[0]
    return np.random.choice(i)

In [ ]:
# Listing 3.15 Estimate p_best with the online bootstrap
def estimate_pbest_ob(obs):
    counts = [0] * len(obs)
    num_samples = 100
    for _ in range(num_samples):
        ctrs = [ob.CTR_estimate() for ob in obs]
        ctrs = np.array(ctrs)
        i = np.where(ctrs == ctrs.max())[0]
        if len(i)==1:
            counts[i[0]] += 1
            
    return np.array(counts)/num_samples

### 3.3.3	The complete algorithm

In [ ]:
# Listing 3.16 Thompson sampling
def thompson_sampling():
    k = 4
    num_bs_means = 100
    p_stop = 0.95
    smallest_sum_difference = 1
    prac_sig = 0.001

    min_samples_per_arm = smallest_sum_difference / prac_sig
    
    obs = [OnlineBootstrap(num_bs_means) for _ in range(k)]
    sum_clicks = 0.0
    num_ads = 0.0
    ctr_vs_n = []

    n = 0
    while True:
        num_samples_per_arm = [ob.count() for ob in obs]
        i_too_few = np.where(np.array(num_samples_per_arm) < min_samples_per_arm)[0]
        if len(i_too_few) > 0:
            i_selected = np.random.choice(i_too_few)
        else:
            i_selected = rpm_select_arm_ob(obs)
        i_clicked = measure_arm(i_selected)
        obs[i_selected].append(i_clicked)
        sum_clicks += i_clicked
        num_ads += 1
        ctr_vs_n.append(sum_clicks / num_ads)

        n += 1
        if len(i_too_few) == 0 and n % 100 == 0:
            p_bests = estimate_pbest_ob(obs)
            i_best_arm = np.where(p_bests == p_bests.max())[0]
            if len(i_best_arm) == 1 and p_bests.max() >= p_stop:
                break
                
    return ctr_vs_n, i_best_arm        
    

In [ ]:
np.random.seed(17)
ctr_ts, i_best_arm = thompson_sampling()
print (len(ctr_ts), ctr_ts[-1], i_best_arm)

In [ ]:
np.random.seed(17)
mean_selected, std_selected = run_multi(lambda: [thompson_sampling()[1]==3], 100)
print (mean_selected, std_selected)

In [ ]:
def run_multi_count(trace_fn, num=100):
    nums = []
    for _ in range(num):
        trace = trace_fn()
        nums.append(len(trace))

    nums = np.array(nums)
    
    return nums.mean(), nums.max()

In [ ]:
np.random.seed(17)
nums_mean, nums_max = run_multi_count(lambda: thompson_sampling()[0], 100)
print (nums_mean, nums_max)

In [ ]:
def thompson_sampling_compare():
    k = 4
    num_bs_means = 100
    p_stop = 0.95
    smallest_sum_difference = 1
    prac_sig = 0.001

    min_samples_per_arm = smallest_sum_difference / prac_sig
    
    obs = [OnlineBootstrap(num_bs_means) for _ in range(k)]
    sum_clicks = 0.0
    num_ads = 0.0
    ctr_vs_n = []

    n = 0
    accepted = False
    for _ in range(200000):
        if accepted:
            i_selected = i_best_arm
        else:
            num_samples_per_arm = [ob.count() for ob in obs]
            i_too_few = np.where(np.array(num_samples_per_arm) < min_samples_per_arm)[0]
            if len(i_too_few) > 0:
                i_selected = np.random.choice(i_too_few)
            else:
                i_selected = rpm_select_arm_ob(obs)
                
        i_clicked = measure_arm(i_selected)
        obs[i_selected].append(i_clicked)
        sum_clicks += i_clicked
        num_ads += 1
        ctr_vs_n.append(sum_clicks / num_ads)

        n += 1
        if not accepted and len(i_too_few) == 0 and n % 100 == 0:
            p_bests = estimate_pbest_ob(obs)
            i_best_arm = np.where(p_bests == p_bests.max())[0]
            if len(i_best_arm) == 1 and p_bests.max() >= p_stop:
                i_best_arm = i_best_arm[0]
                accepted = True
                
    return ctr_vs_n, i_best_arm        
    

In [ ]:
np.random.seed(17)
mean_ts_compare, std_ts_compare = run_multi(lambda: thompson_sampling_compare()[0], 100)
print (mean_ts_compare[-1], std_ts_compare[-1])

In [ ]:
mean_egd_tr = mean_egd
std_egd_tr = std_egd

means_ts_tr = mean_ts_compare
stds_ts_tr = std_ts_compare

n = np.arange(len(mean_egd_tr))[::100]

plt.fill_between(n,
                 (mean_egd_tr-std_egd_tr/2)[::100],
                 (mean_egd_tr+std_egd_tr/2)[::100],
                 color=e4e.color_2, alpha=.75, linewidth=1)
plt.plot(n, mean_egd_tr[::100], '--', color=e4e.color_2)

nts = np.arange(len(means_ts_tr))[::100]  # TODO
plt.fill_between(nts,
                 (means_ts_tr-stds_ts_tr/2)[::100],
                 (means_ts_tr+stds_ts_tr/2)[::100],
                 color=e4e.color_3, alpha=.75, linewidth=1)
plt.plot(nts, means_ts_tr[::100], '-', color=e4e.color_1)

c = plt.axis()
plt.xlabel('n, index to individual measurement')
plt.ylabel('CTR up through n');
plt.legend(['epsilon-greedy', 'Thompson sampling'], loc='lower right')
plt.axis([c[0], c[1], .007, .011]);

e4e.save_fig(18)